<a href="https://colab.research.google.com/github/Ryan-R-Davis/Baseball-Steals/blob/main/Scrape_RE_Matrixes_from_BP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
!pip install MechanicalSoup
import mechanicalsoup
import time
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/Portfolio/Baseball Steals/Clean Data.csv'



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
years = range(1950, 2020)
url = "https://legacy.baseballprospectus.com/sortable/index.php?cid=2800999"
browser = mechanicalsoup.Browser()
page = browser.get(url)
if page.status_code == 200:
  print("Page loaded sucessfully")
else:
  print("Page failed to load")

Page loaded sucessfully


In [3]:
cleanData = []
cleanRow = []
cols = ["Year", "BaseOuts", "ExpectedRuns"]

for year in years:
  #print(year)
  time.sleep(.5) # small delay in order to keep from overloading the server
  url = "https://legacy.baseballprospectus.com/sortable/index.php?cid=2800999"
  browser = mechanicalsoup.Browser()
  page = browser.get(url)
  form = page.soup.select("form")[2]
  form.select("option")[0]["value"] = year
  page = browser.submit(form, page.url)
  Tables = page.soup.find_all("table")
  for i in range(9,63,7): # Selects desired columns from one/each row of the raw table
    row = Tables[4].find_all("td")[i:i+5]
    for ii in range(0,3): # for each Base+Outs situation creates an individual row
      cleanRow.append(row[0].text) # starts row with the year
      cleanRow.append(row[1].text + f"Outs{ii}") # combines the base and outs into one variable, BaseOuts. adds it to the row
      cleanRow.append(float(row[ii+2].text)) # adds the numeric value for the expected runs in that situation to the row. 
      #print(cleanRow)
      cleanData.append(cleanRow)
      cleanRow = []

df = pd.DataFrame(cleanData, columns=cols)
df.set_index('Year', inplace=True)
print(df)

1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
      BaseOuts  ExpectedRuns
Year                        
1950  000Outs0        0.5556
1950  000Outs1        0.3078
1950  000Outs2        0.1131
1950  003Outs0        1.3618
1950  003Outs1        0.9663
...        ...           ...
2019  120Outs1        0.9792
2019  120Outs2        0.4666
2019  123Outs0        2.3617
2019  123Outs1        1.6337
2019  123Outs2        0.7426

[1680 rows x 2 columns]


In [5]:
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)